In [21]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [22]:
F_selection = pd.read_csv("F_Selection/top_N_features_Mutual_Information_ScoreAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [23]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [24]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_11288\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [25]:
top = top_10

rank_data_10 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest): {mse_rf}')
    print(f'R-squared (RandomForest): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 6.101965941295546
R-squared (RandomForest): 0.019640536607300585


,Player_name,Rank,Pos,Points_won,Predicted_points
13506,Tom Chambers,-1.0,PF,0.0,12.630
13866,Sean Higgins,-1.0,SF,0.0,10.585
13836,Michael Jordan,4,SG,6.0,8.945
13903,Karl Malone,-1.0,PF,0.0,6.270
13889,Tim Duncan,5T,PF,4.0,6.020
13764,Gary Payton,2,PG,37.0,3.160
13605,David Robinson,3,C,10.0,3.140
13532,Todd Day,-1.0,SG,0.0,2.555
13631,Derrick McKey,-1,SF,0.0,2.530
13555,Latrell Sprewell,-1,SG,0.0,2.510


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 15.46501408227848
R-squared (RandomForest): 0.08951641476031424


,Player_name,Rank,Pos,Points_won,Predicted_points
13434,David Robinson,4,C,3.0,5.935
13269,Alonzo Mourning,1,C,89.0,5.090
13121,Bo Outlaw,-1,PF,0.0,5.055
13039,Hakeem Olajuwon,7T,C,1.0,4.635
13397,Scottie Pippen,7T,SF,1.0,4.530
13406,Dennis Rodman,-1,PF,0.0,4.470
13050,Charles Oakley,-1,PF,0.0,4.470
13087,Horace Grant,-1,PF,0.0,4.465
13237,Eddie Jones,7T,SF,1.0,4.455
13380,Mookie Blaylock,-1,PG,0.0,4.430


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 12.986855555555557
R-squared (RandomForest): -0.33940701706636367


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1184,Nikola Jokić,-1.0,C,0.0,175.245
1022,Rudy Gobert,3.0,C,136.0,115.105
1007,Joel Embiid,-1,C,0.0,67.945
713,Marcus Smart,1,PG,257.0,11.365
678,Chris Paul,-1,PG,0.0,9.970
981,Robert Williams,7.0,C,8.0,8.145
1212,Jayson Tatum,-1.0,SF,0.0,7.610
933,Al Horford,9,C,3.0,7.155
1306,Bam Adebayo,4.0,C,128.0,7.030
992,Paul George,-1.0,PF,0.0,5.280


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 367.46512569786535
R-squared (RandomForest): 0.13642701207632457


,Player_name,Rank,Pos,Points_won,Predicted_points
497,Joel Embiid,9T,C,7.0,45.915
137,Evan Mobley,3.0,PF,101.0,42.390
524,Bam Adebayo,5.0,C,18.0,33.260
206,Nic Claxton,9T,C,7.0,32.255
408,Jarrett Allen,-1,C,0.0,32.145
323,Brook Lopez,2.0,C,309.0,31.965
141,Nikola Vučević,-1.0,C,0.0,29.560
191,Jayson Tatum,-1.0,SF,0.0,25.660
59,Jaren Jackson Jr.,1.0,C,391.0,20.065
357,Nikola Jokić,-1.0,C,0.0,16.335


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1.0,4,-1,8T,1,4,10T,10,-1,-1,...,1.0,2,1,2.0,3,1.0,2,1.0,-1.0,9T
1,-1.0,1,-1,-1,-1,13T,-1,1,13T,-1,...,13T,8,2,1,1.0,3.0,3.0,4.0,3.0,3.0
2,4,-1,-1,-1,-1,8,7,8,-1,2,...,7.0,1,5.0,-1,15T,2.0,-1,2.0,-1,5.0
3,-1.0,7T,2,5T,-1,1,-1,5,6,13T,...,10.0,4,3,3,15T,-1,1.0,7,1,9T
4,5T,7T,-1,-1,-1,-1,6,4,1,3,...,8T,10T,10,-1,14.0,-1,6T,3,-1,-1
5,2,-1,8T,-1,-1,3,2,9,3,6,...,13T,7.0,4.0,-1,6,-1,-1.0,6.0,7.0,2.0
6,3,-1,-1,-1,-1,9,1,-1,7,4,...,4.0,-1.0,-1.0,-1,-1.0,-1,8T,-1,-1.0,-1.0
7,-1.0,-1,3T,3,-1,-1,10T,-1,4,1,...,11,5.0,-1,5T,8.0,6T,-1,-1,9,-1.0
8,-1,7T,8T,2,-1,-1,-1,-1,10T,20.0,...,-1,15T,11T,5T,-1,-1,-1,5.0,4.0,1.0
9,-1,-1,5T,5T,-1,-1,9,-1,5,-1.0,...,-1,13T,-1,5T,-1.0,-1.0,-1,9.0,-1.0,-1.0


In [26]:
top = top_20

rank_data_20 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest): {mse_rf}')
    print(f'R-squared (RandomForest): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 8.122177327935223
R-squared (RandomForest): -0.3049324567525076


,Player_name,Rank,Pos,Points_won,Predicted_points
13942,Glen Rice,-1.0,SF,0.0,13.870
13948,Grant Hill,-1.0,SF,0.0,13.865
13568,Antoine Walker,-1.0,PF,0.0,13.815
13714,Ray Allen,-1.0,SG,0.0,13.815
13979,Michael Finley,-1.0,SF,0.0,13.815
13506,Tom Chambers,-1.0,PF,0.0,11.540
13866,Sean Higgins,-1.0,SF,0.0,10.680
13836,Michael Jordan,4,SG,6.0,7.315
13601,Kevin Garnett,11T,PF,1.0,6.565
13889,Tim Duncan,5T,PF,4.0,5.075


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 15.77724214135021
R-squared (RandomForest): 0.07113437377906384


,Player_name,Rank,Pos,Points_won,Predicted_points
13434,David Robinson,4,C,3.0,7.365
13121,Bo Outlaw,-1,PF,0.0,4.945
13087,Horace Grant,-1,PF,0.0,4.620
13269,Alonzo Mourning,1,C,89.0,4.480
13397,Scottie Pippen,7T,SF,1.0,4.435
13406,Dennis Rodman,-1,PF,0.0,4.415
13050,Charles Oakley,-1,PF,0.0,4.405
13237,Eddie Jones,7T,SF,1.0,4.355
13488,Tim Duncan,5T,PF,2.0,4.110
13424,Gary Payton,3,PG,6.0,4.075


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 11.039718002136752
R-squared (RandomForest): -0.13858783561894095


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1184,Nikola Jokić,-1.0,C,0.0,149.080
1046,Sekou Doumbouya,-1.0,PF,0.0,108.065
610,Michael Porter Jr.,-1.0,SF,0.0,94.080
1022,Rudy Gobert,3.0,C,136.0,88.940
1007,Joel Embiid,-1,C,0.0,67.375
992,Paul George,-1.0,PF,0.0,34.545
1212,Jayson Tatum,-1.0,SF,0.0,32.765
1275,Stephen Curry,-1.0,PG,0.0,30.190
813,Ricky Rubio,-1.0,PG,0.0,27.485
762,Jaylen Brown,-1.0,SF,0.0,25.570


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 429.73629462233174
R-squared (RandomForest): -0.00991530900101334


,Player_name,Rank,Pos,Points_won,Predicted_points
76,Mac McClung,-1.0,SG,0.0,98.690
141,Nikola Vučević,-1.0,C,0.0,29.935
191,Jayson Tatum,-1.0,SF,0.0,17.120
137,Evan Mobley,3.0,PF,101.0,13.225
497,Joel Embiid,9T,C,7.0,9.480
357,Nikola Jokić,-1.0,C,0.0,7.975
301,Anthony Davis,-1,C,0.0,6.645
524,Bam Adebayo,5.0,C,18.0,6.390
176,Giannis Antetokounmpo,6.0,PF,14.0,6.040
323,Brook Lopez,2.0,C,309.0,5.220


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1.0,4,8T,8T,-1,4,2,4,13T,-1.0,...,1.0,2,1,2.0,3,1.0,2,1.0,-1.0,-1.0
1,-1.0,-1,2,3,-1,6,4,5,-1,-1,...,11,1,2,1,1.0,3.0,1.0,-1.0,-1.0,-1.0
2,-1.0,-1,-1,2,-1,8,7,8,-1,-1,...,13T,4,3,-1,15T,-1,3.0,-1.0,-1.0,-1.0
3,-1.0,1,7,-1,2,13T,-1,-1,-1.0,3,...,5,5.0,5.0,-1,-1.0,-1,8T,7,3.0,3.0
4,-1.0,7T,5T,5T,1,3,1,1,6,13T,...,7.0,-1.0,10,-1.0,15T,2.0,8T,3,-1,9T
5,-1.0,-1,-1,-1,-1,-1,6,9,1,6,...,10.0,10T,-1.0,-1.0,14.0,-1,-1,-1,-1.0,-1.0
6,-1.0,-1,-1,-1,3T,9,-1,-1,7,4,...,4.0,8,-1,-1.0,2,-1,6T,-1,-1.0,-1
7,4,7T,-1,11T,3T,1,-1.0,10,-1,2,...,13T,15T,-1.0,-1.0,-1.0,6T,-1,5.0,-1.0,5.0
8,11T,5T,3T,-1.0,-1,-1,-1,11,4,-1.0,...,8T,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,6.0
9,5T,3,-1,-1,-1,-1,-1,7,4,16T,...,-1.0,7.0,-1,5T,-1.0,-1,10.0,6.0,-1.0,2.0


In [27]:
top = top_30

rank_data_30 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest): {mse_rf}')
    print(f'R-squared (RandomForest): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 7.519623329959514
R-squared (RandomForest): -0.20812439197409738


,Player_name,Rank,Pos,Points_won,Predicted_points
13506,Tom Chambers,-1.0,PF,0.0,12.185
13942,Glen Rice,-1.0,SF,0.0,11.500
13948,Grant Hill,-1.0,SF,0.0,11.490
13979,Michael Finley,-1.0,SF,0.0,11.490
13714,Ray Allen,-1.0,SG,0.0,11.490
13568,Antoine Walker,-1.0,PF,0.0,11.490
13866,Sean Higgins,-1.0,SF,0.0,11.005
13836,Michael Jordan,4,SG,6.0,7.785
13903,Karl Malone,-1.0,PF,0.0,6.225
13889,Tim Duncan,5T,PF,4.0,4.890


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 15.809090770042193
R-squared (RandomForest): 0.0692593251381659


,Player_name,Rank,Pos,Points_won,Predicted_points
13434,David Robinson,4,C,3.0,7.200
13269,Alonzo Mourning,1,C,89.0,4.010
13121,Bo Outlaw,-1,PF,0.0,3.980
13406,Dennis Rodman,-1,PF,0.0,3.930
13087,Horace Grant,-1,PF,0.0,3.755
13182,Dikembe Mutombo,2,C,10.0,3.740
13237,Eddie Jones,7T,SF,1.0,3.705
13050,Charles Oakley,-1,PF,0.0,3.695
13380,Mookie Blaylock,-1,PG,0.0,3.580
13488,Tim Duncan,5T,PF,2.0,3.470


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 10.412014957264958
R-squared (RandomForest): -0.07384931139815332


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1184,Nikola Jokić,-1.0,C,0.0,132.820
1046,Sekou Doumbouya,-1.0,PF,0.0,103.355
610,Michael Porter Jr.,-1.0,SF,0.0,90.520
1022,Rudy Gobert,3.0,C,136.0,68.390
1007,Joel Embiid,-1,C,0.0,56.150
992,Paul George,-1.0,PF,0.0,51.090
1275,Stephen Curry,-1.0,PG,0.0,35.165
762,Jaylen Brown,-1.0,SF,0.0,28.175
813,Ricky Rubio,-1.0,PG,0.0,27.950
1212,Jayson Tatum,-1.0,SF,0.0,25.860


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 435.00676453201976
R-squared (RandomForest): -0.02230134274782758


,Player_name,Rank,Pos,Points_won,Predicted_points
76,Mac McClung,-1.0,SG,0.0,99.355
184,Shaquille Harrison,-1.0,SG,0.0,60.985
392,Jacob Gilyard,-1.0,PG,0.0,41.405
141,Nikola Vučević,-1.0,C,0.0,19.840
191,Jayson Tatum,-1.0,SF,0.0,14.450
357,Nikola Jokić,-1.0,C,0.0,11.835
323,Brook Lopez,2.0,C,309.0,9.310
137,Evan Mobley,3.0,PF,101.0,8.790
497,Joel Embiid,9T,C,7.0,7.680
176,Giannis Antetokounmpo,6.0,PF,14.0,6.035


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1.0,4,2,8T,-1,4,7,4,13T,13T,...,1.0,2,1,2.0,3,1.0,1.0,1.0,-1.0,-1.0
1,-1.0,1,8T,3,1,3,2,10,-1,3,...,7.0,4,2,1,6,2.0,8T,-1.0,-1.0,-1.0
2,-1.0,-1,-1,2,-1,6,6,5,-1,-1,...,11,1,3,5T,15T,-1,3.0,-1.0,-1.0,-1.0
3,-1.0,-1,3T,-1,-1,1,-1,1,6,-1,...,5,3.0,10,-1,-1.0,3.0,2,7,3.0,-1.0
4,-1.0,-1,5T,5T,2,13T,4,8,7,-1.0,...,10.0,-1.0,4.0,-1.0,1.0,4,-1,-1,-1,-1.0
5,-1.0,2,-1,-1,-1,8,-1.0,9,1,1,...,13T,-1.0,5.0,-1,-1.0,-1.0,8T,3,-1.0,-1.0
6,-1.0,7T,7,-1,3T,-1,1,11,-1,16T,...,-1.0,5.0,-1.0,5T,-1.0,-1,8T,5.0,-1.0,2.0
7,4,-1,-1,-1.0,3T,9,-1,7,-1.0,6,...,13T,10T,-1.0,-1.0,15T,-1,10.0,-1,-1.0,3.0
8,-1.0,-1,-1,11T,-1,-1,5,-1,-1,4,...,8T,15T,-1,-1,-1.0,-1,-1.0,6.0,-1.0,9T
9,5T,5T,-1,1,-1,-1,-1,-1.0,-1.0,-1.0,...,-1.0,7.0,11T,-1.0,-1.0,-1.0,-1.0,-1,-1.0,6.0


In [28]:
top = top_40

rank_data_40 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest): {mse_rf}')
    print(f'R-squared (RandomForest): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 7.047398532388665
R-squared (RandomForest): -0.13225539276939324


,Player_name,Rank,Pos,Points_won,Predicted_points
13942,Glen Rice,-1.0,SF,0.0,11.675
13948,Grant Hill,-1.0,SF,0.0,11.630
13568,Antoine Walker,-1.0,PF,0.0,11.535
13714,Ray Allen,-1.0,SG,0.0,11.535
13979,Michael Finley,-1.0,SF,0.0,11.535
13506,Tom Chambers,-1.0,PF,0.0,7.440
13836,Michael Jordan,4,SG,6.0,7.325
13866,Sean Higgins,-1.0,SF,0.0,5.855
13903,Karl Malone,-1.0,PF,0.0,5.260
13889,Tim Duncan,5T,PF,4.0,4.960


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 11.675393248945149
R-squared (RandomForest): 0.3126256563475025


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,16.215
13237,Eddie Jones,7T,SF,1.0,5.420
13424,Gary Payton,3,PG,6.0,5.270
13434,David Robinson,4,C,3.0,4.995
13380,Mookie Blaylock,-1,PG,0.0,4.550
13182,Dikembe Mutombo,2,C,10.0,4.290
13406,Dennis Rodman,-1,PF,0.0,4.155
13345,Kendall Gill,-1,SF,0.0,3.905
13039,Hakeem Olajuwon,7T,C,1.0,3.845
13488,Tim Duncan,5T,PF,2.0,3.745


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 6.117573931623931
R-squared (RandomForest): 0.36906039984912786


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1046,Sekou Doumbouya,-1.0,PF,0.0,108.535
1184,Nikola Jokić,-1.0,C,0.0,106.895
610,Michael Porter Jr.,-1.0,SF,0.0,94.130
1022,Rudy Gobert,3.0,C,136.0,54.775
1026,Myles Turner,-1.0,C,0.0,52.545
992,Paul George,-1.0,PF,0.0,41.110
1212,Jayson Tatum,-1.0,SF,0.0,35.715
997,Jaren Jackson Jr.,5.0,PF,99.0,29.530
1007,Joel Embiid,-1,C,0.0,27.605
1275,Stephen Curry,-1.0,PG,0.0,27.155


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 328.4901318144499
R-squared (RandomForest): 0.22802142354132415


,Player_name,Rank,Pos,Points_won,Predicted_points
184,Shaquille Harrison,-1.0,SG,0.0,70.925
76,Mac McClung,-1.0,SG,0.0,70.760
59,Jaren Jackson Jr.,1.0,C,391.0,62.240
392,Jacob Gilyard,-1.0,PG,0.0,55.285
323,Brook Lopez,2.0,C,309.0,46.930
206,Nic Claxton,9T,C,7.0,45.370
137,Evan Mobley,3.0,PF,101.0,33.210
75,Myles Turner,-1.0,C,0.0,23.005
208,Walker Kessler,-1.0,C,0.0,22.520
497,Joel Embiid,9T,C,7.0,21.725


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1.0,1,1,3,-1,1,2,-1,1,13T,...,1.0,4,3,2.0,15T,1.0,1.0,1.0,-1.0,-1.0
1,-1.0,7T,7,2,1,4,-1,6.0,13T,3,...,11,2,1,-1.0,3,2.0,3.0,9.0,-1.0,-1.0
2,-1.0,3,5T,8T,-1,3,6,9,7,-1.0,...,5,1,2,5T,1.0,-1,2,-1.0,-1.0,1.0
3,-1.0,4,2,-1,-1,6,3,4,-1,1,...,7.0,-1.0,10,1,6,5.0,11,-1.0,3.0,-1.0
4,-1.0,-1,3T,5T,2,13T,7,11,-1,-1,...,13T,7.0,4.0,-1,-1.0,-1,10.0,-1,-1.0,2.0
5,-1.0,2,-1,-1,-1,8,-1.0,7,-1.0,-1,...,10.0,15T,11T,-1.0,14.0,-1.0,8T,-1,-1.0,9T
6,4,-1,-1,-1,3T,-1,5,5,6,16T,...,-1.0,3.0,-1.0,-1.0,-1,-1.0,-1,7,-1.0,3.0
7,-1.0,-1,-1,1,-1.0,9,-1,10,-1,4,...,-1.0,5.0,5.0,5T,-1,4,8T,5.0,5.0,-1.0
8,-1.0,7T,3T,1,-1,13T,-1,-1.0,-1,24T,...,4.0,-1,-1,-1,15T,-1,-1.0,-1,-1,-1.0
9,5T,5T,8T,-1,-1,11,10T,-1.0,3,6,...,13T,-1.0,-1.0,-1.0,-1.0,-1.0,8T,-1,-1.0,9T


In [29]:
top = top_50

rank_data_50 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest): {mse_rf}')
    print(f'R-squared (RandomForest): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV', 'GP_espn', 'PF', 'GP_nba', 'G', 'teamates_with_dpoy', 'DEF RTG_nba', 'pace_impact_raptor', 'BLK_100_poss', 'D-LEBRON_bball', 'PW_misc']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest): 6.615935779352227
R-squared (RandomForest): -0.06293

,Player_name,Rank,Pos,Points_won,Predicted_points
13942,Glen Rice,-1.0,SF,0.0,8.840
13948,Grant Hill,-1.0,SF,0.0,8.840
13714,Ray Allen,-1.0,SG,0.0,8.750
13568,Antoine Walker,-1.0,PF,0.0,8.750
13979,Michael Finley,-1.0,SF,0.0,8.705
13506,Tom Chambers,-1.0,PF,0.0,8.390
13866,Sean Higgins,-1.0,SF,0.0,7.305
13836,Michael Jordan,4,SG,6.0,6.975
13889,Tim Duncan,5T,PF,4.0,5.685
13601,Kevin Garnett,11T,PF,1.0,5.375


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV', 'GP_espn', 'PF', 'GP_nba', 'G', 'teamates_with_dpoy', 'DEF RTG_nba', 'pace_impact_raptor', 'BLK_100_poss', 'D-LEBRON_bball', 'PW_misc']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Erro

,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,16.125
13237,Eddie Jones,7T,SF,1.0,5.480
13434,David Robinson,4,C,3.0,5.155
13182,Dikembe Mutombo,2,C,10.0,5.045
13424,Gary Payton,3,PG,6.0,4.575
13488,Tim Duncan,5T,PF,2.0,4.070
13406,Dennis Rodman,-1,PF,0.0,3.940
13380,Mookie Blaylock,-1,PG,0.0,3.875
13445,Kevin Garnett,7T,PF,1.0,3.730
13087,Horace Grant,-1,PF,0.0,3.700


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV', 'GP_espn', 'PF', 'GP_nba', 'G', 'teamates_with_dpoy', 'DEF RTG_nba', 'pace_impact_raptor', 'BLK_100_poss', 'D-LEBRON_bball', 'PW_misc']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Erro

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1046,Sekou Doumbouya,-1.0,PF,0.0,101.470
1184,Nikola Jokić,-1.0,C,0.0,92.655
1022,Rudy Gobert,3.0,C,136.0,87.570
610,Michael Porter Jr.,-1.0,SF,0.0,87.200
1026,Myles Turner,-1.0,C,0.0,45.080
992,Paul George,-1.0,PF,0.0,37.355
933,Al Horford,9,C,3.0,31.075
813,Ricky Rubio,-1.0,PG,0.0,27.550
1212,Jayson Tatum,-1.0,SF,0.0,26.170
1007,Joel Embiid,-1,C,0.0,25.715


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_votes', 'DWS_advanced', 'war_total_raptor', 'predator_defense_raptor', 'war_reg_season_raptor', 'WS_advanced', 'predator_total_raptor', 'raptor_total_raptor', 'poss_raptor', 'VORP_advanced', 'DEF WS_nba', 'mp_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP', 'GS', 'MP_100_poss', 'BPM_advanced', 'MIN_nba', 'OPP PTS PAINT_nba', 'MPG_espn', 'WINS_espn', 'RPM_espn', 'OPP PTS 2ND CHANCE_nba', 'TRB', 'OPP PTS FB_nba', 'DRB', 'OPP PTS OFF TOV_nba', 'STL_nba', 'DREB_nba', 'BLK', 'DBPM_advanced', 'BLK_nba', 'WS/48_advanced', 'W_nba', 'STL', 'DRPM_espn', 'PER_advanced', 'DRtg_100_poss', 'TOV', 'GP_espn', 'PF', 'GP_nba', 'G', 'teamates_with_dpoy', 'DEF RTG_nba', 'pace_impact_raptor', 'BLK_100_poss', 'D-LEBRON_bball', 'PW_misc']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Erro

,Player_name,Rank,Pos,Points_won,Predicted_points
76,Mac McClung,-1.0,SG,0.0,70.825
59,Jaren Jackson Jr.,1.0,C,391.0,57.490
323,Brook Lopez,2.0,C,309.0,48.285
206,Nic Claxton,9T,C,7.0,44.825
184,Shaquille Harrison,-1.0,SG,0.0,37.255
137,Evan Mobley,3.0,PF,101.0,25.255
392,Jacob Gilyard,-1.0,PG,0.0,21.575
208,Walker Kessler,-1.0,C,0.0,20.630
497,Joel Embiid,9T,C,7.0,15.715
75,Myles Turner,-1.0,C,0.0,14.835


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1.0,1,1,3,1,1,2,-1,1,3,...,1.0,4,3,2.0,15T,1.0,1.0,1.0,-1.0,-1.0
1,-1.0,7T,7,2,-1,4,-1,6.0,13T,13T,...,7.0,2,1,-1.0,1.0,2.0,3.0,9.0,-1.0,1.0
2,-1.0,4,-1,8T,-1,3,6,4,7,-1.0,...,5,1,2,1,6,-1,11,-1.0,3.0,2.0
3,-1.0,2,3T,-1,2,6,-1.0,9,-1,-1,...,11,-1.0,5.0,5T,3,5.0,2,-1.0,-1.0,9T
4,-1.0,3,5T,5T,-1,8,3,11,-1,1,...,10.0,3.0,4.0,-1.0,-1.0,-1,-1.0,-1,-1.0,-1.0
5,-1.0,5T,2,-1,-1,13T,7,7,-1.0,-1,...,13T,5.0,-1,-1,15T,-1.0,10.0,6.0,-1.0,3.0
6,-1.0,-1,3T,-1,3T,9,5,10,6,16T,...,4.0,15T,-1.0,-1.0,8.0,-1.0,-1,3,9,-1.0
7,4,-1,-1,1,-1.0,-1,-1,5,-1,4,...,-1.0,7.0,10,-1,-1.0,4,-1.0,-1,-1.0,-1.0
8,5T,7T,8T,1,-1,11,-1,-1.0,-1,6,...,13T,-1,11T,-1.0,-1.0,-1.0,-1.0,-1,-1.0,9T
9,11T,-1,-1,-1,-1,11,-1.0,-1.0,3,24T,...,-1.0,-1.0,-1.0,5T,-1,-1,8T,10,-1,-1.0


In [30]:
rank_data_10.to_csv('RDF+MI_Period_rank_data_10.csv', index=False)
rank_data_20.to_csv('RDF+MI_Period_rank_data_20.csv', index=False)
rank_data_30.to_csv('RDF+MI_Period_rank_data_30.csv', index=False)
rank_data_40.to_csv('RDF+MI_Period_rank_data_40.csv', index=False)
rank_data_50.to_csv('RDF+MI_Period_rank_data_50.csv', index=False)